In [5]:
import abacus_cosmos.Halos as ach
from astropy.table import Table
import matplotlib.pyplot as plt
import astropy.cosmology as apc
import os
import numpy as np
import glob
from astropy import constants as const
from collections import Counter
%matplotlib inline

In [6]:
dtype=[('BoxID','i8'), ('hubble', 'f8'), ('omega_de', 'f8'),
      ('omega_m', 'f8'), ('n_s', 'f8'), ('sigma_8', 'f8'), ('w_0', 'f8')]
cosmo_data = np.loadtxt("../abacus/box_cosmo_params.dat", dtype=dtype)

In [21]:
def get_sphere(box_id=0, RSD=False, standard_cosmology=True, redshift='z0.100'):
    main_path = "/Users/forero/data/AbacusCosmos/"
    
    if standard_cosmology:
        halo_path = "AbacusCosmos_720box_planck_00_{:02d}_FoF_halos_{}".format(box_id, redshift)
    else:
        halo_path = "AbacusCosmos_720box_{:02d}_FoF_halos_{}".format(box_id, redshift)

    halo_data = Table(ach.read_halos_FoF(os.path.join(main_path,halo_path)))

    vcirc_max_limit = 300.0
    r_limit = 300
    r_limit_in = 250
    #hubble = cosmo_data['hubble'][box_id]/100
       
    halo_data['r_comov'] = np.sqrt(np.sum(halo_data['pos']**2, axis=1))
    halo_data['v_radial'] = np.sum(halo_data['pos']*halo_data['vel'], axis=1)/halo_data['r_comov']

    if RSD:
        halo_data['r'] = halo_data['r_comov'] + halo_data['v_radial']/100
    else:
        halo_data['r'] = halo_data['r_comov'].copy()
        
    #halo_data['r'] = halo_data['r']/hubble

    ii = (halo_data['r']<r_limit) & (halo_data['vcirc_max']>vcirc_max_limit) & (halo_data['r']>0)
    print(box_id, 'sphere', np.count_nonzero(ii))
    
    if RSD:
        halo_data['pos'][:,0] = halo_data['pos'][:,0] + (halo_data['pos'][:,0]/halo_data['r_comov'])*(halo_data['v_radial']/100)
        halo_data['pos'][:,1] = halo_data['pos'][:,1] + (halo_data['pos'][:,1]/halo_data['r_comov'])*(halo_data['v_radial']/100)
        halo_data['pos'][:,2] = halo_data['pos'][:,2] + (halo_data['pos'][:,2]/halo_data['r_comov'])*(halo_data['v_radial']/100)

        
    sphere_data = halo_data['pos'][ii]
    n_points = len(sphere_data)
    ii_shell = (halo_data['r']>r_limit_in) & (halo_data['r']<r_limit) & (halo_data['vcirc_max']>vcirc_max_limit) 
    shell_data = halo_data['pos'][ii_shell]
    print(box_id, 'shell', np.count_nonzero(ii_shell))

    
    random_pos = sphere_data.copy()
    phi  = np.random.random(n_points)* np.pi * 2.0
    theta = np.arccos(2.0*(np.random.random(n_points)-0.5))
    r = np.sqrt(np.sum(sphere_data**2, axis=1))
    random_pos[:,0] = r * np.cos(phi) * np.sin(theta)
    random_pos[:,1] = r * np.sin(phi) * np.sin(theta)
    random_pos[:,2] = r * np.cos(theta)
    
    if RSD:
        rsd_name = 'rsd'
    else:
        rsd_name = 'norsd'
        
    if standard_cosmology:
        cosmo_name = 'stdcosmo'
    else:
        cosmo_name = 'nonstdcosmo'
    
    shell_filename = "../abacus/shell_{:02d}_{}_{}_{}.dat".format(box_id, cosmo_name, rsd_name, redshift)
    output_filename = "../abacus/sphere_{:02d}_{}_{}_{}.dat".format(box_id, cosmo_name, rsd_name, redshift)
    random_filename = "../abacus/random_sphere_{:02d}_{}_{}_{}.dat".format(box_id, cosmo_name, rsd_name, redshift)

    np.savetxt(output_filename, sphere_data)
    np.savetxt(random_filename, random_pos)
    np.savetxt(shell_filename, shell_data)
    print('writing to', output_filename)
    print('writing to', random_filename)
    print('writing to', shell_filename)

    del halo_data
    del sphere_data
    del random_pos

In [24]:
for i in range(20):
    get_sphere(box_id=i, RSD=True, standard_cosmology=True, redshift='z0.100')
    get_sphere(box_id=i, RSD=False, standard_cosmology=True, redshift='z0.100')

0 sphere 83738
0 shell 34965
writing to ../abacus/sphere_00_stdcosmo_rsd_z0.100.dat
writing to ../abacus/random_sphere_00_stdcosmo_rsd_z0.100.dat
writing to ../abacus/shell_00_stdcosmo_rsd_z0.100.dat
0 sphere 83216
0 shell 34802
writing to ../abacus/sphere_00_stdcosmo_norsd_z0.100.dat
writing to ../abacus/random_sphere_00_stdcosmo_norsd_z0.100.dat
writing to ../abacus/shell_00_stdcosmo_norsd_z0.100.dat
1 sphere 81370
1 shell 35155
writing to ../abacus/sphere_01_stdcosmo_rsd_z0.100.dat
writing to ../abacus/random_sphere_01_stdcosmo_rsd_z0.100.dat
writing to ../abacus/shell_01_stdcosmo_rsd_z0.100.dat
1 sphere 81629
1 shell 35013
writing to ../abacus/sphere_01_stdcosmo_norsd_z0.100.dat
writing to ../abacus/random_sphere_01_stdcosmo_norsd_z0.100.dat
writing to ../abacus/shell_01_stdcosmo_norsd_z0.100.dat
2 sphere 84450
2 shell 34601
writing to ../abacus/sphere_02_stdcosmo_rsd_z0.100.dat
writing to ../abacus/random_sphere_02_stdcosmo_rsd_z0.100.dat
writing to ../abacus/shell_02_stdcosmo_rsd

In [25]:
for i in range(40):
    get_sphere(box_id=i, RSD=True, standard_cosmology=False, redshift='z0.100')
    get_sphere(box_id=i, RSD=False, standard_cosmology=False, redshift='z0.100')

0 sphere 75389
0 shell 31464
writing to ../abacus/sphere_00_nonstdcosmo_rsd_z0.100.dat
writing to ../abacus/random_sphere_00_nonstdcosmo_rsd_z0.100.dat
writing to ../abacus/shell_00_nonstdcosmo_rsd_z0.100.dat
0 sphere 75022
0 shell 31483
writing to ../abacus/sphere_00_nonstdcosmo_norsd_z0.100.dat
writing to ../abacus/random_sphere_00_nonstdcosmo_norsd_z0.100.dat
writing to ../abacus/shell_00_nonstdcosmo_norsd_z0.100.dat
1 sphere 83193
1 shell 34871
writing to ../abacus/sphere_01_nonstdcosmo_rsd_z0.100.dat
writing to ../abacus/random_sphere_01_nonstdcosmo_rsd_z0.100.dat
writing to ../abacus/shell_01_nonstdcosmo_rsd_z0.100.dat
1 sphere 82843
1 shell 34875
writing to ../abacus/sphere_01_nonstdcosmo_norsd_z0.100.dat
writing to ../abacus/random_sphere_01_nonstdcosmo_norsd_z0.100.dat
writing to ../abacus/shell_01_nonstdcosmo_norsd_z0.100.dat
2 sphere 62789
2 shell 26210
writing to ../abacus/sphere_02_nonstdcosmo_rsd_z0.100.dat
writing to ../abacus/random_sphere_02_nonstdcosmo_rsd_z0.100.dat


In [27]:
for z in ['z0.300', 'z0.500', 'z0.700', 'z1.000', 'z1.500']:
    get_sphere(box_id=0, RSD=True, standard_cosmology=True, redshift=z)
    get_sphere(box_id=0, RSD=False, standard_cosmology=True, redshift=z)

0 sphere 90942
0 shell 37967
writing to ../abacus/sphere_00_stdcosmo_rsd_z0.300.dat
writing to ../abacus/random_sphere_00_stdcosmo_rsd_z0.300.dat
writing to ../abacus/shell_00_stdcosmo_rsd_z0.300.dat
0 sphere 90342
0 shell 37778
writing to ../abacus/sphere_00_stdcosmo_norsd_z0.300.dat
writing to ../abacus/random_sphere_00_stdcosmo_norsd_z0.300.dat
writing to ../abacus/shell_00_stdcosmo_norsd_z0.300.dat
0 sphere 97320
0 shell 40603
writing to ../abacus/sphere_00_stdcosmo_rsd_z0.500.dat
writing to ../abacus/random_sphere_00_stdcosmo_rsd_z0.500.dat
writing to ../abacus/shell_00_stdcosmo_rsd_z0.500.dat
0 sphere 96683
0 shell 40386
writing to ../abacus/sphere_00_stdcosmo_norsd_z0.500.dat
writing to ../abacus/random_sphere_00_stdcosmo_norsd_z0.500.dat
writing to ../abacus/shell_00_stdcosmo_norsd_z0.500.dat
0 sphere 102006
0 shell 42541
writing to ../abacus/sphere_00_stdcosmo_rsd_z0.700.dat
writing to ../abacus/random_sphere_00_stdcosmo_rsd_z0.700.dat
writing to ../abacus/shell_00_stdcosmo_rs